#Telecom Domain Read & Write Ops Assignment - Building Datalake & Lakehouse
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS telecom_catalog_assign;
CREATE SCHEMA IF NOT EXISTS telecom_catalog_assign.landing_zone;
CREATE VOLUME IF NOT EXISTS telecom_catalog_assign.landing_zone.landing_vol;

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
# customer_csv = [
#     (101, "Arun", 31, "Chennai", "PREPAID"),
#     (102, "Meera", 45, "Bangalore", "POSTPAID"),
#     (103, "Irfan", 29, "Hyderabad", "PREPAID"),
#     (104,"Raj", 52, "Mumbai", "POSTPAID"),
#     (105,None, 27, "Delhi", "PREPAID"),
#     (106, "Sneha", None, "Pune", "PREPAID")
# ]
# schema = ["id", "name", "age","city","plan"]
# df = spark.createDataFrame( data = customer_csv, schema = schema )
# df.show()




##2. Filesystem operations
1. Write dbutils.fs code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
customer_csv = """id,name,age,city,plan
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
"""


usage_tsv = """customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
"""

tower_logs_region1 = """event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
"""
dbutils.fs.put("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer1.csv", customer_csv, True)
dbutils.fs.put("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.csv", usage_tsv, True)
dbutils.fs.put("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv", tower_logs_region1, True)


try:
    dbutils.fs.ls("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.csv")
    dbutils.fs.ls("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv")
    dbutils.fs.ls("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
    print("File exists and copy successful")
except:
    print("File not found or copy failed")

##3. Spark Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
#1
df1_different_subdirs=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/",inferSchema=True,recursiveFileLookup=True,pathGlobFilter="tower*")
df1_different_subdirs.count()

In [0]:
#2
spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/",inferSchema=True,recursiveFileLookup=True,pathGlobFilter="tower*").count()  

In [0]:
#3
df2 = spark.read \
    .option("inferSchema", True) \
    .option("header", True) \
    .option("recursiveFileLookup", True) \
    .option("pathGlobFilter", "cust*") \
    .csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/")

df2.count()

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
#1
df2 = spark.read \
    .option("inferSchema", False) \
    .option("header", False) \
    .csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
df2.show()

In [0]:
#1
df2 = spark.read \
    .option("inferSchema", True) \
    .option("header", True) \
    .csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
df2.show()

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
#1
df2 = spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv").toDF("id","fname","age","Location","plan_details")
df2.show()

In [0]:
#2
from pyspark.sql.types import *

schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("voice_mins", IntegerType(), True),
    StructField("data_mb", IntegerType(), True),
    StructField("sms_count", IntegerType(), True)
])

df = spark.read \
    .schema(schema) \
    .option("header", True) \
    .csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.csv")

df.show()

## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

# df1 = spark.read.option("header", True) \
#     .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
# df1.write.mode("overwrite") \
#   .option("header", True) \
#   .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
# df2 = spark.read.option("header", True) \
#     .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.csv")
# df2.write.mode("append") \
#   .option("header", True) \
#   .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/")
df3 = spark.read.option("header", True) \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower.csv")
df3.write.mode("overwrite") \
  .option("delimiter", "|") \
  .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower.csv")

df3.show(5)

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
df.write.json("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.json")

In [0]:
df.write.json("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.json", mode="append",compression="snappy")

In [0]:
df.write.json("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower.json", mode="ignore",compression="snappy")
display(df)

##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()
df=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
df.write.parquet("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.parquet",mode="overwrite",compression="gzip")

# df1.write.parquet("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.parquet",mode="overwrite",compression="gzip")

In [0]:
#3
df=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv")
df.write.parquet("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.parquet",compression="gzip")

In [0]:
#2
df=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.csv")
df.write.parquet("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.parquet",mode="error")
df.show(5)

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
df=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
df.write.orc("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.orc",mode="overwrite")
df1=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.csv")
df1.write.orc("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage.orc",mode="append")
df2=spark.read.csv("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.csv")
df2.write.orc("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1.orc",mode="overwrite")

df1.show(5)

##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
df.write.format("delta").save("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer",mode='overwrite')
df1.write.format("delta").save("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/cusage/usage",mode="append")
df2.write.format("delta").save("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_region1",mode='overwrite')
df2.show(5)

##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


In [0]:
df.write.saveAsTable("default.customer",mode='overwrite')#default delta format
spark.read.table("default.customer").show(2)
df1.write.saveAsTable("default.usage",mode='overwrite')#default delta format
spark.read.table("default.usage").show(2)

##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

In [0]:
df.write.insertInto("customer",overwrite=True)

##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
df.write.xml("dbfs:/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer1",rowTag='customer',mode='overwrite')

##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

##15. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables
